In [13]:
import tensorflow as tf
tf.enable_eager_execution()
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [14]:
import numpy as np
import pandas as pd
import glob
import cv2
import random
import imgaug as ia
import imgaug.augmenters as iaa

from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.layers import Dense, Input, Dropout, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

from collections import Counter
from tensorflow.keras.applications.nasnet import preprocess_input

In [15]:
"""
    CREATE AUGMENTER PIPELINE
    wsi_dataset.py
"""
def get_seq():
    sometimes = lambda aug: iaa.Sometimes(0.5, aug)
    seq = iaa.Sequential(
        [
            # apply the following augmenters to most images
            iaa.Fliplr(0.5),  # horizontally flip 50% of all images
            iaa.Flipud(0.2),  # vertically flip 20% of all images
            sometimes(iaa.Affine(
                scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
                # scale images to 80-120% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},  # translate by -20 to +20 percent (per axis)
                rotate=(-10, 10),  # rotate by -45 to +45 degrees
                shear=(-5, 5),  # shear by -16 to +16 degrees
                order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
                cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
                mode=ia.ALL  # use any of scikit-image's warping modes (see 2nd image from the top for examples)
            )),
            # execute 0 to 5 of the following (less important) augmenters per image
            # don't execute all of them, as that would often be way too strong
            iaa.SomeOf((0, 5),
                       [
                           sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
                           # convert images into their superpixel representation
                           iaa.OneOf([
                               iaa.GaussianBlur((0, 1.0)),  # blur images with a sigma between 0 and 3.0
                               iaa.AverageBlur(k=(3, 5)),
                               # blur image using local means with kernel sizes between 2 and 7
                               iaa.MedianBlur(k=(3, 5)),
                               # blur image using local medians with kernel sizes between 2 and 7
                           ]),
                           iaa.Sharpen(alpha=(0, 1.0), lightness=(0.9, 1.1)),  # sharpen images
                           iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),  # emboss images
                           # search either for all edges or for directed edges,
                           # blend the result with the original image using a blobby mask
                           iaa.SimplexNoiseAlpha(iaa.OneOf([
                               iaa.EdgeDetect(alpha=(0.5, 1.0)),
                               iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                           ])),
                           iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01 * 255), per_channel=0.5),
                           # add gaussian noise to images
                           iaa.OneOf([
                               iaa.Dropout((0.01, 0.05), per_channel=0.5),  # randomly remove up to 10% of the pixels
                               iaa.CoarseDropout((0.01, 0.03), size_percent=(0.01, 0.02), per_channel=0.2),
                           ]),
                           iaa.Invert(0.01, per_channel=True),  # invert color channels
                           iaa.Add((-2, 2), per_channel=0.5),
                           # change brightness of images (by -10 to 10 of original value)
                           iaa.AddToHueAndSaturation((-1, 1)),  # change hue and saturation
                           # either change the brightness of the whole image (sometimes
                           # per channel) or change the brightness of subareas
                           iaa.OneOf([
                               iaa.Multiply((0.9, 1.1), per_channel=0.5),
                               iaa.FrequencyNoiseAlpha(
                                   exponent=(-1, 0),
                                   first=iaa.Multiply((0.9, 1.1), per_channel=True),
                                   second=iaa.ContrastNormalization((0.9, 1.1))
                               )
                           ]),
                           sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),
                           # move pixels locally around (with random strengths)
                           sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),
                           # sometimes move parts of the image around
                           sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                       ],
                       random_order=True
                       )
        ],
        random_order=True
    )
    return seq
seq = get_seq()

In [16]:
"""
    CREATE NASNET MODEL
    nasnet_model.py
"""
def get_model_classif_nasnet():
    inputs = Input((299, 299, 3))
    base_model = NASNetMobile(include_top=False, input_tensor=inputs, weights='imagenet')
    x = base_model(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    out = Dense(1, activation="sigmoid", name="3_")(out)
    model = Model(inputs, out)
    model.compile(optimizer=Adam(0.0001), loss=binary_crossentropy, metrics=['acc', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    model.summary()

    return model
model = get_model_classif_nasnet()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 10, 10, 1056) 4269716     input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
flatten_1 

In [17]:
"""
    WEIGHTED FILENAME SAMPLING
    wsi_dataset.py
"""
DATASET_DIR_TRAIN = '/home/matejg/Project/crc_ml/data/processed/train_slides/'
DATASET_DIR_VALID = '/home/matejg/Project/crc_ml/data/processed/valid_slides/'
DATASET_DIR_TEST = '/home/matejg/Project/crc_ml/data/processed/test_slides/'

def get_filenames(data_type='train'):
    if data_type == 'train':
        DATASET_DIR = DATASET_DIR_TRAIN
    elif data_type == 'valid':
        DATASET_DIR = DATASET_DIR_VALID
    elif data_type == 'test':
        DATASET_DIR = DATASET_DIR_TEST
    
    all_tiles = []
    all_labels = []
    all_weights = []

    for label_id, label_name in {'0': 'normal', '1': 'cancer'}.items():

        slides = glob.glob(DATASET_DIR + '/*/{}'.format(label_name))
        print('Found {} {} slides.'.format(len(slides), label_name))
        per_slide_proba = 0.5 / len(slides)
        print('Each {} slide will be selected with proba {:.4f}'.format(label_name, per_slide_proba))

        for slide in slides:
            slide_tiles = glob.glob(slide + '/*w299-h299.png')
            print('   - found {} tiles.'.format(len(slide_tiles)))
            slide_labels = [label_id] * len(slide_tiles)
            slide_weights = [per_slide_proba / len(slide_tiles)] * len(slide_tiles)

            all_tiles += slide_tiles
            all_labels += slide_labels
            all_weights += slide_weights
    pd_data = pd.DataFrame({'filename': all_tiles, 'class': all_labels})
    return pd_data, all_weights

In [18]:
"""
    AUGMENTERS FOR TEST DATASET
    wsi_dataset.py
"""
def get_test_augmenter(augment_type):
    seq = None
    if augment_type == 'h':
        print('Horizontal augment.')
        seq = iaa.Sequential([iaa.Fliplr(1)])
    elif augment_type == 'v':
        print('Vertical augment')
        seq = iaa.Sequential([iaa.Flipud(1)])
    elif augment_type == 'b':
        print('Horizontal+Vertical augment')
        seq = iaa.Sequential([iaa.Flipud(1), iaa.Fliplr(1)])
    return seq

"""
    UTILITY CHUNKER
    wsi_dataset.py
"""
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


"""
    SHORTCUT UTILITY FUNCTION
    wsi_dataset.py
"""
def divide_round_up(n, d):
    return (n + (d - 1))//d


"""
    TEST DATA GENERATOR
    wsi_dataset.py
"""
def test_data_gen(pd_data, batch_size, augment=None, shuffle=True):
    if augment in ['h','v','b']:
        seq = get_test_augmenter(augment)
    else:
        seq = get_seq()
    
    while True:
        if shuffle:
            pd_data = pd_data.sample(frac=1)
            
        for batch in chunker(pd_data, batch_size):
            data = [cv2.imread(x) for x in batch['filename']]
            if augment != 'n':
                data = seq.augment_images(data)
            data = [preprocess_input(x) for x in data]
            yield np.array(data), np.array(batch['class'])
            

"""
    TRAIN DATA GENERATOR
    wsi_dataset.py
"""
def data_gen(pd_data, weights, batch_size):
    while True:
        batch = pd_data.sample(n=batch_size, replace=True, weights=weights)
        data = [cv2.imread(x) for x in batch['filename']]
        data = seq.augment_images(data)
        data = [preprocess_input(x) for x in data]
        yield np.array(data), np.array(batch['class'].astype(np.int))

In [19]:
train_fn, train_w = get_filenames('train')
valid_fn, valid_w = get_filenames('valid')
test_fn, test_w = get_filenames('test')

Found 11 cancer slides.
Each cancer slide will be selected with proba 0.0455
   - found 582 tiles.
   - found 4815 tiles.
   - found 618 tiles.
   - found 1144 tiles.
   - found 343 tiles.
   - found 2823 tiles.
   - found 626 tiles.
   - found 2789 tiles.
   - found 3283 tiles.
   - found 621 tiles.
   - found 567 tiles.
Found 36 normal slides.
Each normal slide will be selected with proba 0.0139
   - found 3105 tiles.
   - found 7526 tiles.
   - found 3479 tiles.
   - found 4450 tiles.
   - found 5101 tiles.
   - found 6929 tiles.
   - found 5661 tiles.
   - found 6291 tiles.
   - found 10695 tiles.
   - found 4887 tiles.
   - found 3804 tiles.
   - found 4823 tiles.
   - found 21844 tiles.
   - found 16419 tiles.
   - found 3369 tiles.
   - found 7216 tiles.
   - found 6878 tiles.
   - found 3903 tiles.
   - found 45182 tiles.
   - found 2950 tiles.
   - found 4949 tiles.
   - found 5501 tiles.
   - found 4725 tiles.
   - found 6974 tiles.
   - found 18677 tiles.
   - found 4447 til

In [ ]:
new_train_fn = train_fn[train_fn['class'] == '0'].sample(n=len(train_fn[train_fn['class'] == '1']))
new_train_fn = pd.concat([new_train_fn, train_fn[train_fn['class'] == '1']])
new_train_fn = new_train_fn.sample(frac=1)

new_valid_fn = valid_fn[valid_fn['class'] == '0'].sample(n=len(valid_fn[valid_fn['class'] == '1']))
new_valid_fn = pd.concat([new_valid_fn, valid_fn[valid_fn['class'] == '1']])
new_valid_fn = new_valid_fn.sample(frac=1)

new_test_fn = test_fn[test_fn['class'] == '0'].sample(n=len(test_fn[test_fn['class'] == '1']))
new_test_fn = pd.concat([new_test_fn, valid_fn[test_fn['class'] == '1']])
new_test_fn = new_test_fn.sample(frac=1)

In [8]:
batch_size = 32

In [ ]:
checkpoint = ModelCheckpoint('/home/matejg/nasnet-wsi.h5', monitor='val_precision', verbose=1, save_best_only=True, mode='max', save_weights_only=True)

In [ ]:
_ = model.fit_generator(
    test_data_gen(new_train_fn, batch_size),
    validation_data=test_data_gen(valid_fn, batch_size, shuffle=False),
    epochs=6, verbose=1,
    callbacks=[checkpoint],
    steps_per_epoch=divide_round_up(len(new_train_fn), batch_size),
    validation_steps=divide_round_up(len(valid_fn), batch_size)
)

In [12]:
if True:
    used_ds = test_fn
    #model.load_weights('/home/matejg/nasnet-wsi.h5')
    experiment_id = 'f81c0d8e-03c2-40f9-89ee-15bedbd89a57'
    model.load_weights('/home/matejg/Project/crc_ml/models/checkpoints/{eid}/{eid}.hdf5'.format(eid=experiment_id))

    res_normal = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='n', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_horizontal = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='h', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_vertical = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='v', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_both = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='b', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )
    
    res_combined = np.array((res_normal.ravel() * res_horizontal.ravel() * res_vertical.ravel() * res_both.ravel()) ** 0.25)
    for res in [res_normal, res_vertical, res_horizontal, res_both, res_combined]:
        acc = tf.keras.metrics.BinaryAccuracy()
        acc.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())

        p = tf.keras.metrics.Precision()
        p.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())

        r = tf.keras.metrics.Recall()
        r.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())
        
        print('Acc: {:.3f}\tP: {:.3f}\tR: {:.3f}'.format(acc.result().numpy(), p.result().numpy(), r.result().numpy()))

500/500 [==============================] - 95s 191ms/step
Horizontal augment.
500/500 [==============================] - 96s 192ms/step
Vertical augment
500/500 [==============================] - 89s 177ms/step
Horizontal+Vertical augment
500/500 [==============================] - 95s 189ms/step
Acc: 0.710	P: 0.674	R: 0.845
Acc: 0.722	P: 0.695	R: 0.820
Acc: 0.716	P: 0.684	R: 0.832
Acc: 0.721	P: 0.693	R: 0.822
Acc: 0.727	P: 0.699	R: 0.825


In [ ]:
for guess, truth in zip(res_combined, test_fn['class'].ravel()):
    if ((guess < 0.5) and (truth == '1')) or (guess >= 0.5 and truth == '0'):
        print('{:.5f}\t{}'.format(guess, truth))

In [8]:
all_runs = {0: (0.91153276, 0.90771484, 0.9162149),
 1: (0.92237556, 0.9302209, 0.9132578),
 2: (0.9078364, 0.94133335, 0.86988664),
 3: (0.90832925, 0.8795236, 0.9462789),
 4: (0.92336124, 0.92609125, 0.92015773),
 5: (0.9070971, 0.9171717, 0.89502215),
 6: (0.9098078, 0.9138875, 0.9048793),
 7: (0.9103006, 0.9258312, 0.89206505),
 8: (0.9132578, 0.911231, 0.915722),
 9: (0.91695416, 0.90988374, 0.92557913)}

In [11]:
all_runs

{0: (0.91153276, 0.90771484, 0.9162149),
 1: (0.92237556, 0.9302209, 0.9132578),
 2: (0.9078364, 0.94133335, 0.86988664),
 3: (0.90832925, 0.8795236, 0.9462789),
 4: (0.92336124, 0.92609125, 0.92015773),
 5: (0.9070971, 0.9171717, 0.89502215),
 6: (0.9098078, 0.9138875, 0.9048793),
 7: (0.9103006, 0.9258312, 0.89206505),
 8: (0.9132578, 0.911231, 0.915722),
 9: (0.91695416, 0.90988374, 0.92557913),
 10: (0.9073435, 0.9017987, 0.91424346),
 11: (0.90857565, 0.9400212, 0.87284374),
 12: (0.8824544, 0.9394111, 0.8176442),
 13: (0.91670775, 0.92253876, 0.9098078),
 14: (0.915722, 0.89806515, 0.9379004),
 15: (0.91670775, 0.9047391, 0.93149334),
 16: (0.91202563, 0.94849783, 0.8713652),
 17: (0.9159685, 0.90851885, 0.92508626),
 18: (0.92286843, 0.9359756, 0.9078364),
 19: (0.8925579, 0.9612044, 0.818137),
 20: (0.90685064, 0.9244216, 0.88615084),
 21: (0.9137506, 0.92985153, 0.89502215),
 22: (0.919172, 0.9297625, 0.90685064),
 23: (0.9107935, 0.91160494, 0.9098078),
 24: (0.90832925, 0.92

In [10]:
for i in range(10,100):
    model = get_model_classif_nasnet()
    new_train_fn = train_fn[train_fn['class'] == '0'].sample(n=len(train_fn[train_fn['class'] == '1']))
    new_train_fn = pd.concat([new_train_fn, train_fn[train_fn['class'] == '1']])
    new_train_fn = new_train_fn.sample(frac=1)

    new_valid_fn = valid_fn[valid_fn['class'] == '0'].sample(n=len(valid_fn[valid_fn['class'] == '1']))
    new_valid_fn = pd.concat([new_valid_fn, valid_fn[valid_fn['class'] == '1']])
    new_valid_fn = new_valid_fn.sample(frac=1)

    new_test_fn = test_fn[test_fn['class'] == '0'].sample(n=len(test_fn[test_fn['class'] == '1']))
    new_test_fn = pd.concat([new_test_fn, valid_fn[test_fn['class'] == '1']])
    new_test_fn = new_test_fn.sample(frac=1)

    batch_size = 32
    checkpoint = ModelCheckpoint('/home/matejg/nasnet-wsi.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max', save_weights_only=True)

    _ = model.fit_generator(
        test_data_gen(new_train_fn, batch_size),
        validation_data=test_data_gen(new_valid_fn, batch_size, shuffle=False),
        epochs=6, verbose=1,
        callbacks=[checkpoint],
        steps_per_epoch=divide_round_up(len(new_train_fn), batch_size),
        validation_steps=divide_round_up(len(new_valid_fn), batch_size)
    )

    used_ds = new_test_fn
    model.load_weights('/home/matejg/nasnet-wsi.h5')

    res_normal = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='n', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_horizontal = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='h', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_vertical = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='v', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_both = model.predict_generator(
        test_data_gen(used_ds, batch_size, augment='b', shuffle=False),
        verbose=1, steps=divide_round_up(len(used_ds), batch_size)
    )

    res_combined = np.array((res_normal.ravel() * res_horizontal.ravel() * res_vertical.ravel() * res_both.ravel()) ** 0.25)
    for res in [res_normal, res_vertical, res_horizontal, res_both, res_combined]:
        acc = tf.keras.metrics.BinaryAccuracy()
        acc.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())

        p = tf.keras.metrics.Precision()
        p.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())

        r = tf.keras.metrics.Recall()
        r.update_state(used_ds['class'].ravel().astype(np.float), res.ravel())

        print('Acc: {:.3f}\tP: {:.3f}\tR: {:.3f}'.format(acc.result().numpy(), p.result().numpy(), r.result().numpy()))
        all_runs[i] = (acc.result().numpy(), p.result().numpy(), r.result().numpy())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 10, 10, 1056) 4269716     input_1[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d (GlobalMax (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
global_average_poolin

Epoch 1/6
127/127 [==============================] - 70s 552ms/step - loss: 0.5654 - acc: 0.7647 - precision_6: 0.8048 - recall_6: 0.6989

Epoch 00001: val_acc improved from -inf to 0.76466, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1137s 998ms/step - loss: 0.4998 - acc: 0.7674 - precision_6: 0.7729 - recall_6: 0.7575 - val_loss: 0.5654 - val_acc: 0.7647 - val_precision_6: 0.8048 - val_recall_6: 0.6989
Epoch 2/6
127/127 [==============================] - 68s 534ms/step - loss: 0.5264 - acc: 0.7750 - precision_6: 0.8980 - recall_6: 0.6205

Epoch 00002: val_acc improved from 0.76466 to 0.77501, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 986s 866ms/step - loss: 0.3600 - acc: 0.8372 - precision_6: 0.8468 - recall_6: 0.8233 - val_loss: 0.5264 - val_acc: 0.7750 - val_precision_6: 0.8980 - val_recall_6: 0.6205
Epoch 3/6
127/127 [==============================] - 68s 536ms/step - loss: 0.3082 - acc: 0.86

1139/1139 [==============================] - 1289s 1s/step - loss: 0.4997 - acc: 0.7704 - precision_24: 0.7786 - recall_24: 0.7558 - val_loss: 0.8850 - val_acc: 0.6865 - val_precision_24: 0.6190 - val_recall_24: 0.9704
Epoch 2/6
127/127 [==============================] - 68s 535ms/step - loss: 0.4590 - acc: 0.8172 - precision_24: 0.8960 - recall_24: 0.7176

Epoch 00002: val_acc improved from 0.68655 to 0.81715, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1000s 878ms/step - loss: 0.3641 - acc: 0.8356 - precision_24: 0.8462 - recall_24: 0.8202 - val_loss: 0.4590 - val_acc: 0.8172 - val_precision_24: 0.8960 - val_recall_24: 0.7176
Epoch 3/6
127/127 [==============================] - 70s 552ms/step - loss: 0.3146 - acc: 0.8620 - precision_24: 0.8380 - recall_24: 0.8975

Epoch 00003: val_acc improved from 0.81715 to 0.86200, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1002s 879ms/step - loss: 0.3196 - ac

Acc: 0.915	P: 0.899	R: 0.934
Acc: 0.910	P: 0.893	R: 0.931
Acc: 0.917	P: 0.905	R: 0.931
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 10, 10, 1056) 4269716     input_7[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_6 (GlobalM (None, 1056)         0           NASNet[1][0]                     
__________________________________________________________________________________________________
global_average_pooling2d_6 (Glo (None, 1056)         0           NASNet[1][0]                     
______________________

Epoch 1/6
127/127 [==============================] - 143s 1s/step - loss: 0.4640 - acc: 0.7898 - precision_54: 0.8879 - recall_54: 0.6634

Epoch 00001: val_acc improved from -inf to 0.78980, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1572s 1s/step - loss: 0.5053 - acc: 0.7692 - precision_54: 0.7743 - recall_54: 0.7598 - val_loss: 0.4640 - val_acc: 0.7898 - val_precision_54: 0.8879 - val_recall_54: 0.6634
Epoch 2/6
127/127 [==============================] - 73s 574ms/step - loss: 0.3427 - acc: 0.8526 - precision_54: 0.8682 - recall_54: 0.8314

Epoch 00002: val_acc improved from 0.78980 to 0.85264, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1025s 900ms/step - loss: 0.3585 - acc: 0.8369 - precision_54: 0.8476 - recall_54: 0.8215 - val_loss: 0.3427 - val_acc: 0.8526 - val_precision_54: 0.8682 - val_recall_54: 0.8314
Epoch 3/6
127/127 [==============================] - 71s 557ms/step - loss: 0.3098 - a

...

1139/1139 [==============================] - 1876s 2s/step - loss: 0.5096 - acc: 0.7689 - precision_72: 0.7740 - recall_72: 0.7595 - val_loss: 0.6384 - val_acc: 0.7201 - val_precision_72: 0.8530 - val_recall_72: 0.5318
Epoch 2/6
127/127 [==============================] - 70s 554ms/step - loss: 0.3765 - acc: 0.8305 - precision_72: 0.8328 - recall_72: 0.8270

Epoch 00002: val_acc improved from 0.72006 to 0.83046, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1041s 914ms/step - loss: 0.3629 - acc: 0.8380 - precision_72: 0.8478 - recall_72: 0.8240 - val_loss: 0.3765 - val_acc: 0.8305 - val_precision_72: 0.8328 - val_recall_72: 0.8270
Epoch 3/6
127/127 [==============================] - 72s 569ms/step - loss: 0.3341 - acc: 0.8610 - precision_72: 0.8547 - recall_72: 0.8699

Epoch 00003: val_acc improved from 0.83046 to 0.86102, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1044s 916ms/step - loss: 0.3181 - ac

Epoch 1/6
127/127 [==============================] - 210s 2s/step - loss: 0.5532 - acc: 0.7541 - precision_84: 0.8273 - recall_84: 0.6422

Epoch 00001: val_acc improved from -inf to 0.75407, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1948s 2s/step - loss: 0.5046 - acc: 0.7719 - precision_84: 0.7767 - recall_84: 0.7632 - val_loss: 0.5532 - val_acc: 0.7541 - val_precision_84: 0.8273 - val_recall_84: 0.6422
Epoch 2/6
127/127 [==============================] - 75s 588ms/step - loss: 0.3758 - acc: 0.8521 - precision_84: 0.8712 - recall_84: 0.8265

Epoch 00002: val_acc improved from 0.75407 to 0.85214, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1047s 919ms/step - loss: 0.3605 - acc: 0.8398 - precision_84: 0.8491 - recall_84: 0.8265 - val_loss: 0.3758 - val_acc: 0.8521 - val_precision_84: 0.8712 - val_recall_84: 0.8265
Epoch 3/6
127/127 [==============================] - 71s 558ms/step - loss: 0.3364 - a

1139/1139 [==============================] - 2175s 2s/step - loss: 0.5083 - acc: 0.7668 - precision_102: 0.7724 - recall_102: 0.7565 - val_loss: 0.5826 - val_acc: 0.7575 - val_precision_102: 0.8131 - val_recall_102: 0.6688
Epoch 2/6
127/127 [==============================] - 75s 591ms/step - loss: 0.4419 - acc: 0.8300 - precision_102: 0.8945 - recall_102: 0.7482

Epoch 00002: val_acc improved from 0.75752 to 0.82997, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1042s 915ms/step - loss: 0.3636 - acc: 0.8357 - precision_102: 0.8483 - recall_102: 0.8175 - val_loss: 0.4419 - val_acc: 0.8300 - val_precision_102: 0.8945 - val_recall_102: 0.7482
Epoch 3/6
127/127 [==============================] - 74s 581ms/step - loss: 0.3191 - acc: 0.8672 - precision_102: 0.8695 - recall_102: 0.8640

Epoch 00003: val_acc improved from 0.82997 to 0.86718, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1037s 910ms/step - loss:

Epoch 2/6
127/127 [==============================] - 73s 572ms/step - loss: 0.4135 - acc: 0.8369 - precision_120: 0.8507 - recall_120: 0.8172

Epoch 00002: val_acc improved from 0.78241 to 0.83687, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1069s 939ms/step - loss: 0.3571 - acc: 0.8406 - precision_120: 0.8508 - recall_120: 0.8261 - val_loss: 0.4135 - val_acc: 0.8369 - val_precision_120: 0.8507 - val_recall_120: 0.8172
Epoch 3/6
127/127 [==============================] - 73s 572ms/step - loss: 0.4085 - acc: 0.8374 - precision_120: 0.7834 - recall_120: 0.9325

Epoch 00003: val_acc improved from 0.83687 to 0.83736, saving model to /home/matejg/nasnet-wsi.h5
1139/1139 [==============================] - 1067s 937ms/step - loss: 0.3189 - acc: 0.8613 - precision_120: 0.8690 - recall_120: 0.8508 - val_loss: 0.4085 - val_acc: 0.8374 - val_precision_120: 0.7834 - val_recall_120: 0.9325
Epoch 4/6
127/127 [==============================] - 74s 580ms/ste

KeyboardInterrupt: 

In [27]:
np.sum(train_fn['class'] == '0')

290453